In [22]:
pip install tensorflow

  Using cached termcolor-1.1.0.tar.gz (3.9 kB)
  Using cached flatbuffers-1.12-py2.py3-none-any.whl (15 kB)
  Using cached tensorflow_estimator-2.9.0-py2.py3-none-any.whl (438 kB)
  Using cached libclang-14.0.6-py2.py3-none-win_amd64.whl (14.2 MB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached absl_py-1.2.0-py3-none-any.whl (123 kB)
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
  Using cached keras-2.9.0-py2.py3-none-any.whl (1.6 MB)
  Using cached tensorboard-2.9.1-py3-none-any.whl (5.8 MB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached tensorboard_data_server-0.6.1-py3-none-any.whl (2.4 kB)
  Using cached tensorboard_plugin_wit-1.8.1-py3-none-any.whl (781 kB)
  Using cached google_auth_oauthlib-0.4.6-py2.py3-none-any.whl (18 kB)
  Using cached requests_oauthlib-1.3.1-py2.py3-none-any.whl (23 kB)
  

In [24]:
pip show tensorflow

Name: tensorflow
Version: 2.9.1
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: c:\users\user\anaconda3\lib\site-packages
Requires: h5py, libclang, tensorboard, tensorflow-io-gcs-filesystem, flatbuffers, astunparse, packaging, termcolor, typing-extensions, six, protobuf, tensorflow-estimator, numpy, opt-einsum, keras-preprocessing, grpcio, setuptools, absl-py, wrapt, google-pasta, gast, keras
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [25]:
pip install keras 

Note: you may need to restart the kernel to use updated packages.


In [17]:
pip install vaderSentiment

  Using cached vaderSentiment-3.3.2-py2.py3-none-any.whl (125 kB)
Note: you may need to restart the kernel to use updated packages.


In [18]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [70]:
import pandas as pd
from nltk import sent_tokenize
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import re

In [103]:
regex1 = "\[[^\]]*\]|\s-\s.*"
regex2 = "\([^\]]*\)|\s-\s.*"
regex3 = "[^A-Za-z\s]"

new_stopwords = stopwords.words('english')
add_stopwords = ["[", "]", "(", ")", ",", "lyrics", "oohwah",
                 "chorus", "!", "?", "``", "oh", "ha", "ah", "-", "yo", "yeah", "uh", "uhh", ":"]
new_stopwords.extend(add_stopwords)

lemma = WordNetLemmatizer()
sid_obj = SentimentIntensityAnalyzer()

def tokenize(str_lyrics):
    split_lyrics = " ".join(str_lyrics.splitlines())
    split_lyrics = re.sub(regex1, '', split_lyrics)
    split_lyrics = re.sub(regex2, '', split_lyrics)
    split_lyrics = re.sub(regex3, '', split_lyrics)
    sentences = sent_tokenize(split_lyrics)
        
    init_words = []

    for sentence in sentences:
        word_tokens = text_to_word_sequence(sentence)

        for word in word_tokens:
            word = word.lower()
            init_words.append(word)
    
    return init_words
            
def remove_stopwords(words_list):
    filtered_words = []
    for word in words_list:
        if word not in new_stopwords:
            filtered_words.append(word)

    return filtered_words

def lemmatizer(tokenzied_list):
    lemmatized_list = []
    for word in tokenzied_list:
        word = lemma.lemmatize(word)
        lemmatized_list.append(word)
    return lemmatized_list

def get_lyric_sentiment(lyrics):
    sentiment = sid_obj.polarity_scores(lyrics)
    return sentiment



In [109]:
data_df = pd.read_csv(r'C:\STUDY\NLP_Toy_Proj\pre_processing\lyrics_sentiment_dataset_final.csv')
data_df.head()
for i in data_df.index.tolist():
    str_lyrics = data_df.loc[i, 'lyrics']
    
    # tokenize
    init_words_list = tokenize(str_lyrics)
    
    # remove stopword
    filtered_word_list = remove_stopwords(init_words_list)
    
    # lemmatizer
    lemmatized_list = lemmatizer(filtered_word_list)
    
    final_str = " ".join(lemmatized_list)
    
    sentiment = get_lyric_sentiment(final_str)
    print("\n%d번 째 인덱스" % i)
    print(sentiment)
    data_df.loc[i, 'neg_sentiment'] = sentiment['neg']
    data_df.loc[i, 'neu_sentiment'] = sentiment['neu']
    data_df.loc[i, 'pos_sentiment'] = sentiment['pos']
    data_df.loc[i,
           'com_sentiment'] = sentiment['compound']

    data_df.loc[i, 'lyrics'] = final_str

# 하나라도 0점이면 out
filtered_df = data_df[(data_df['neg_sentiment'] != 0.000) & (data_df['neu_sentiment'] != 0.000) & (
    data_df['pos_sentiment'] != 0.000) & (data_df['com_sentiment'] != 0.000)]

filtered_df.to_csv("renew_dataset_final.csv", index=False)


0번 째 인덱스
{'neg': 0.158, 'neu': 0.785, 'pos': 0.056, 'compound': -0.7506}

1번 째 인덱스
{'neg': 0.184, 'neu': 0.612, 'pos': 0.204, 'compound': 0.2023}

2번 째 인덱스
{'neg': 0.028, 'neu': 0.583, 'pos': 0.388, 'compound': 0.988}

3번 째 인덱스
{'neg': 0.041, 'neu': 0.721, 'pos': 0.237, 'compound': 0.979}

4번 째 인덱스
{'neg': 0.172, 'neu': 0.457, 'pos': 0.371, 'compound': 0.9604}

5번 째 인덱스
{'neg': 0.076, 'neu': 0.551, 'pos': 0.373, 'compound': 0.9774}

6번 째 인덱스
{'neg': 0.067, 'neu': 0.646, 'pos': 0.286, 'compound': 0.9695}

7번 째 인덱스
{'neg': 0.038, 'neu': 0.623, 'pos': 0.339, 'compound': 0.9871}

8번 째 인덱스
{'neg': 0.184, 'neu': 0.306, 'pos': 0.511, 'compound': 0.9952}

9번 째 인덱스
{'neg': 0.197, 'neu': 0.652, 'pos': 0.151, 'compound': -0.2134}

10번 째 인덱스
{'neg': 0.127, 'neu': 0.73, 'pos': 0.144, 'compound': 0.2373}

11번 째 인덱스
{'neg': 0.149, 'neu': 0.53, 'pos': 0.321, 'compound': 0.9501}

12번 째 인덱스
{'neg': 0.091, 'neu': 0.604, 'pos': 0.306, 'compound': 0.9826}

13번 째 인덱스
{'neg': 0.075, 'neu': 0.573, 'pos': 0.3

233번 째 인덱스
{'neg': 0.037, 'neu': 0.69, 'pos': 0.273, 'compound': 0.9125}

234번 째 인덱스
{'neg': 0.126, 'neu': 0.493, 'pos': 0.381, 'compound': 0.9709}

235번 째 인덱스
{'neg': 0.191, 'neu': 0.312, 'pos': 0.497, 'compound': 0.9936}

236번 째 인덱스
{'neg': 0.019, 'neu': 0.442, 'pos': 0.539, 'compound': 0.995}

237번 째 인덱스
{'neg': 0.054, 'neu': 0.539, 'pos': 0.407, 'compound': 0.9403}

238번 째 인덱스
{'neg': 0.266, 'neu': 0.565, 'pos': 0.168, 'compound': -0.7003}

239번 째 인덱스
{'neg': 0.081, 'neu': 0.492, 'pos': 0.427, 'compound': 0.9918}

240번 째 인덱스
{'neg': 0.23, 'neu': 0.448, 'pos': 0.322, 'compound': 0.8611}

241번 째 인덱스
{'neg': 0.284, 'neu': 0.366, 'pos': 0.35, 'compound': 0.8481}

242번 째 인덱스
{'neg': 0.027, 'neu': 0.515, 'pos': 0.459, 'compound': 0.9812}

243번 째 인덱스
{'neg': 0.269, 'neu': 0.426, 'pos': 0.306, 'compound': 0.4401}

244번 째 인덱스
{'neg': 0.374, 'neu': 0.385, 'pos': 0.241, 'compound': -0.8385}

245번 째 인덱스
{'neg': 0.341, 'neu': 0.485, 'pos': 0.174, 'compound': -0.8677}

246번 째 인덱스
{'neg': 0.124, 

451번 째 인덱스
{'neg': 0.233, 'neu': 0.565, 'pos': 0.201, 'compound': -0.7067}

452번 째 인덱스
{'neg': 0.323, 'neu': 0.561, 'pos': 0.117, 'compound': -0.9661}

453번 째 인덱스
{'neg': 0.183, 'neu': 0.478, 'pos': 0.339, 'compound': 0.9371}

454번 째 인덱스
{'neg': 0.287, 'neu': 0.416, 'pos': 0.298, 'compound': 0.4646}

455번 째 인덱스
{'neg': 0.029, 'neu': 0.364, 'pos': 0.607, 'compound': 0.9966}

456번 째 인덱스
{'neg': 0.253, 'neu': 0.379, 'pos': 0.368, 'compound': 0.9576}

457번 째 인덱스
{'neg': 0.301, 'neu': 0.429, 'pos': 0.27, 'compound': 0.164}

458번 째 인덱스
{'neg': 0.545, 'neu': 0.455, 'pos': 0.0, 'compound': -0.5859}

459번 째 인덱스
{'neg': 0.259, 'neu': 0.528, 'pos': 0.213, 'compound': -0.0716}

460번 째 인덱스
{'neg': 0.272, 'neu': 0.471, 'pos': 0.256, 'compound': -0.6546}

461번 째 인덱스
{'neg': 0.132, 'neu': 0.601, 'pos': 0.267, 'compound': 0.8645}

462번 째 인덱스
{'neg': 0.272, 'neu': 0.478, 'pos': 0.249, 'compound': 0.25}

463번 째 인덱스
{'neg': 0.045, 'neu': 0.236, 'pos': 0.719, 'compound': 0.9983}

464번 째 인덱스
{'neg': 0.182, 

617번 째 인덱스
{'neg': 0.198, 'neu': 0.429, 'pos': 0.373, 'compound': 0.875}

618번 째 인덱스
{'neg': 0.147, 'neu': 0.516, 'pos': 0.337, 'compound': 0.9695}

619번 째 인덱스
{'neg': 0.21, 'neu': 0.524, 'pos': 0.266, 'compound': 0.9111}

620번 째 인덱스
{'neg': 0.086, 'neu': 0.374, 'pos': 0.54, 'compound': 0.9956}

621번 째 인덱스
{'neg': 0.387, 'neu': 0.5, 'pos': 0.113, 'compound': -0.9654}

622번 째 인덱스
{'neg': 0.153, 'neu': 0.597, 'pos': 0.25, 'compound': 0.8708}

623번 째 인덱스
{'neg': 0.0, 'neu': 0.417, 'pos': 0.583, 'compound': 0.6369}

624번 째 인덱스
{'neg': 0.209, 'neu': 0.557, 'pos': 0.233, 'compound': 0.4742}

625번 째 인덱스
{'neg': 0.131, 'neu': 0.564, 'pos': 0.306, 'compound': 0.9562}

626번 째 인덱스
{'neg': 0.092, 'neu': 0.231, 'pos': 0.677, 'compound': 0.9965}

627번 째 인덱스
{'neg': 0.222, 'neu': 0.638, 'pos': 0.14, 'compound': -0.3239}

628번 째 인덱스
{'neg': 0.306, 'neu': 0.559, 'pos': 0.134, 'compound': -0.921}

629번 째 인덱스
{'neg': 0.172, 'neu': 0.518, 'pos': 0.311, 'compound': 0.9719}

630번 째 인덱스
{'neg': 0.134, 'neu':

812번 째 인덱스
{'neg': 0.044, 'neu': 0.641, 'pos': 0.315, 'compound': 0.9963}

813번 째 인덱스
{'neg': 0.18, 'neu': 0.551, 'pos': 0.269, 'compound': 0.8689}

814번 째 인덱스
{'neg': 0.066, 'neu': 0.443, 'pos': 0.491, 'compound': 0.9887}

815번 째 인덱스
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

816번 째 인덱스
{'neg': 0.11, 'neu': 0.509, 'pos': 0.38, 'compound': 0.9834}

817번 째 인덱스
{'neg': 0.133, 'neu': 0.719, 'pos': 0.147, 'compound': 0.1779}

818번 째 인덱스
{'neg': 0.263, 'neu': 0.737, 'pos': 0.0, 'compound': -0.3612}

819번 째 인덱스
{'neg': 0.339, 'neu': 0.244, 'pos': 0.417, 'compound': 0.9103}

820번 째 인덱스
{'neg': 0.449, 'neu': 0.319, 'pos': 0.232, 'compound': -0.9337}

821번 째 인덱스
{'neg': 0.21, 'neu': 0.377, 'pos': 0.413, 'compound': 0.9696}

822번 째 인덱스
{'neg': 0.146, 'neu': 0.427, 'pos': 0.427, 'compound': 0.9721}

823번 째 인덱스
{'neg': 0.25, 'neu': 0.42, 'pos': 0.33, 'compound': 0.9138}

824번 째 인덱스
{'neg': 0.104, 'neu': 0.63, 'pos': 0.266, 'compound': 0.9201}

825번 째 인덱스
{'neg': 0.477, 'neu': 0.215, 'p

...


21075번 째 인덱스
{'neg': 0.254, 'neu': 0.593, 'pos': 0.153, 'compound': -0.9915}

21076번 째 인덱스
{'neg': 0.261, 'neu': 0.623, 'pos': 0.116, 'compound': -0.9707}

21077번 째 인덱스
{'neg': 0.139, 'neu': 0.563, 'pos': 0.298, 'compound': 0.9677}

21078번 째 인덱스
{'neg': 0.253, 'neu': 0.491, 'pos': 0.257, 'compound': 0.1813}

21079번 째 인덱스
{'neg': 0.127, 'neu': 0.711, 'pos': 0.162, 'compound': 0.8402}

21080번 째 인덱스
{'neg': 0.087, 'neu': 0.359, 'pos': 0.554, 'compound': 0.9991}

21081번 째 인덱스
{'neg': 0.132, 'neu': 0.664, 'pos': 0.204, 'compound': 0.9062}

21082번 째 인덱스
{'neg': 0.232, 'neu': 0.539, 'pos': 0.23, 'compound': -0.0258}

21083번 째 인덱스
{'neg': 0.159, 'neu': 0.662, 'pos': 0.18, 'compound': 0.8512}

21084번 째 인덱스
{'neg': 0.184, 'neu': 0.725, 'pos': 0.092, 'compound': -0.9878}

21085번 째 인덱스
{'neg': 0.094, 'neu': 0.75, 'pos': 0.156, 'compound': 0.979}

21086번 째 인덱스
{'neg': 0.065, 'neu': 0.779, 'pos': 0.156, 'compound': 0.9147}

21087번 째 인덱스
{'neg': 0.199, 'neu': 0.73, 'pos': 0.07, 'compound': -0.9249}



21208번 째 인덱스
{'neg': 0.118, 'neu': 0.665, 'pos': 0.218, 'compound': 0.9912}

21209번 째 인덱스
{'neg': 0.0, 'neu': 0.508, 'pos': 0.492, 'compound': 0.9274}

21210번 째 인덱스
{'neg': 0.222, 'neu': 0.53, 'pos': 0.248, 'compound': -0.1134}

21211번 째 인덱스
{'neg': 0.205, 'neu': 0.605, 'pos': 0.189, 'compound': -0.8839}

21212번 째 인덱스
{'neg': 0.387, 'neu': 0.508, 'pos': 0.105, 'compound': -0.9984}

21213번 째 인덱스
{'neg': 0.153, 'neu': 0.657, 'pos': 0.19, 'compound': 0.8667}

21214번 째 인덱스
{'neg': 0.164, 'neu': 0.724, 'pos': 0.112, 'compound': -0.9795}

21215번 째 인덱스
{'neg': 0.273, 'neu': 0.57, 'pos': 0.157, 'compound': -0.9908}

21216번 째 인덱스
{'neg': 0.133, 'neu': 0.741, 'pos': 0.126, 'compound': 0.1531}

21217번 째 인덱스
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

21218번 째 인덱스
{'neg': 0.111, 'neu': 0.8, 'pos': 0.089, 'compound': -0.9428}

21219번 째 인덱스
{'neg': 0.084, 'neu': 0.815, 'pos': 0.101, 'compound': 0.3008}

21220번 째 인덱스
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

21221번 째 인덱스
{'ne

21323번 째 인덱스
{'neg': 0.119, 'neu': 0.512, 'pos': 0.368, 'compound': 0.9951}

21324번 째 인덱스
{'neg': 0.218, 'neu': 0.782, 'pos': 0.0, 'compound': -0.9194}

21325번 째 인덱스
{'neg': 0.265, 'neu': 0.443, 'pos': 0.292, 'compound': -0.1695}

21326번 째 인덱스
{'neg': 0.032, 'neu': 0.621, 'pos': 0.347, 'compound': 0.9696}

21327번 째 인덱스
{'neg': 0.053, 'neu': 0.497, 'pos': 0.45, 'compound': 0.9908}

21328번 째 인덱스
{'neg': 0.09, 'neu': 0.827, 'pos': 0.083, 'compound': -0.1779}

21329번 째 인덱스
{'neg': 0.0, 'neu': 0.842, 'pos': 0.158, 'compound': 0.128}

21330번 째 인덱스
{'neg': 0.28, 'neu': 0.34, 'pos': 0.38, 'compound': 0.9941}

21331번 째 인덱스
{'neg': 0.142, 'neu': 0.638, 'pos': 0.221, 'compound': 0.7606}

21332번 째 인덱스
{'neg': 0.242, 'neu': 0.494, 'pos': 0.265, 'compound': 0.8098}

21333번 째 인덱스
{'neg': 0.189, 'neu': 0.418, 'pos': 0.394, 'compound': 0.9647}

21334번 째 인덱스
{'neg': 0.218, 'neu': 0.718, 'pos': 0.063, 'compound': -0.7343}

21335번 째 인덱스
{'neg': 0.038, 'neu': 0.74, 'pos': 0.222, 'compound': 0.7894}

21336번

21442번 째 인덱스
{'neg': 0.098, 'neu': 0.697, 'pos': 0.205, 'compound': 0.9604}

21443번 째 인덱스
{'neg': 0.291, 'neu': 0.421, 'pos': 0.288, 'compound': 0.5267}

21444번 째 인덱스
{'neg': 0.0, 'neu': 0.429, 'pos': 0.571, 'compound': 0.7181}

21445번 째 인덱스
{'neg': 0.393, 'neu': 0.442, 'pos': 0.165, 'compound': -0.9971}

21446번 째 인덱스
{'neg': 0.229, 'neu': 0.67, 'pos': 0.101, 'compound': -0.8658}

21447번 째 인덱스
{'neg': 0.117, 'neu': 0.658, 'pos': 0.225, 'compound': 0.8481}

21448번 째 인덱스
{'neg': 0.0, 'neu': 0.904, 'pos': 0.096, 'compound': 0.3818}

21449번 째 인덱스
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

21450번 째 인덱스
{'neg': 0.048, 'neu': 0.391, 'pos': 0.561, 'compound': 0.9986}

21451번 째 인덱스
{'neg': 0.195, 'neu': 0.674, 'pos': 0.131, 'compound': -0.8417}

21452번 째 인덱스
{'neg': 0.27, 'neu': 0.542, 'pos': 0.187, 'compound': -0.3698}

21453번 째 인덱스
{'neg': 0.465, 'neu': 0.397, 'pos': 0.138, 'compound': -0.9468}

21454번 째 인덱스
{'neg': 0.032, 'neu': 0.881, 'pos': 0.087, 'compound': 0.6433}

21455번 째 


21695번 째 인덱스
{'neg': 0.209, 'neu': 0.616, 'pos': 0.175, 'compound': -0.9636}

21696번 째 인덱스
{'neg': 0.032, 'neu': 0.822, 'pos': 0.146, 'compound': 0.9791}

21697번 째 인덱스
{'neg': 0.291, 'neu': 0.568, 'pos': 0.141, 'compound': -0.9944}

21698번 째 인덱스
{'neg': 0.075, 'neu': 0.825, 'pos': 0.1, 'compound': 0.4215}

21699번 째 인덱스
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

21700번 째 인덱스
{'neg': 0.366, 'neu': 0.504, 'pos': 0.131, 'compound': -0.9973}

21701번 째 인덱스
{'neg': 0.268, 'neu': 0.702, 'pos': 0.029, 'compound': -0.9569}

21702번 째 인덱스
{'neg': 0.123, 'neu': 0.747, 'pos': 0.13, 'compound': -0.2023}

21703번 째 인덱스
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

21704번 째 인덱스
{'neg': 0.313, 'neu': 0.581, 'pos': 0.105, 'compound': -0.998}

21705번 째 인덱스
{'neg': 0.319, 'neu': 0.6, 'pos': 0.081, 'compound': -0.7845}

21706번 째 인덱스
{'neg': 0.102, 'neu': 0.72, 'pos': 0.178, 'compound': 0.8815}

21707번 째 인덱스
{'neg': 0.223, 'neu': 0.694, 'pos': 0.083, 'compound': -0.9823}

21708번 째 인덱스
{'

In [16]:
df_prev = pd.read_csv(
    r"C:\STUDY\NLP_Toy_Proj\pre_processing\billboard_dataset.csv")

# documents = [TaggedDocument(doc, [i])
#              for i, doc in enumerate(common_texts)]
# print(documents)
for i in range(5):
    listSeries = df_prev.loc[i, 'lyrics']
    print(listSeries)

['i', 'believe', 'lyrics', 'i', 'believe', 'for', 'every', 'drop', 'of', 'rain', 'that', 'falls', 'a', 'flower', 'grows', 'i', 'believe', 'that', 'somewhere', 'in', 'the', 'darkest', 'night', 'a', 'candle', 'glows', 'i', 'believe', 'for', 'everyone', 'who', 'goes', 'astray', 'someone', 'will', 'come', 'to', 'show', 'the', 'way', 'i', 'believe', 'i', 'believe', 'i', 'believe', 'above', 'the', 'storm', 'the', 'smallest', 'prayer', 'will', 'still', 'be', 'heard', 'i', 'believe', 'that', 'someone', 'in', 'the', 'great', 'somewhere', 'hears', 'every', 'word', 'every', 'time', 'i', 'hear', 'a', 'newborn', 'baby', 'cry', 'or', 'touch', 'a', 'leaf', 'or', 'see', 'the', 'sky', 'then', 'i', 'know', 'why', 'i', 'believe', 'every', 'time', 'i', 'hear', 'a', 'newborn', 'baby', 'cry', 'or', 'touch', 'a', 'leaf', 'or', 'see', 'the', 'sky', 'then', 'i', 'know', 'why', 'i', 'believe', 'embed']
['cry', 'lyrics', 'oohwah', 'oohwah', 'oohwah', 'if', 'your', 'sweetheart', 'sends', 'a', 'letter', 'of', 'goo

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from nltk.stem import WordNetLemmatizer
import nltk
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument